In [ ]:
import pandas as pd
import nltk

# Read the CSV file (fixed the quotes)
df = pd.read_csv("C:\\Users\\abcjv\\Downloads\\spotify_reviews.csv", usecols=['content', 'score'])

# Display the DataFrame
df

In [ ]:
# Function to convert rating stars to sentiment
def stars_to_sentiment(stars):
    if stars <= 2:
        return 'negative'
    elif stars == 3:
        return 'neutral'
    else:
        return 'positive'

# Ensure 'score' column exists before applying
if 'score' in df.columns:
    df['sentiment'] = df['score'].apply(stars_to_sentiment)
else:
    print("Error: Column 'score' not found in DataFrame.")

# Correct way to select multiple columns
df


In [ ]:
import re

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords

# Ensure necessary NLTK resources are downloaded
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Expand contractions
    contractions = {
        "isn't": "is not", "aren't": "are not", "can't": "cannot", "couldn't": "could not",
        "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
        "hasn't": "has not", "haven't": "have not", "hadn't": "had not", "isn't": "is not",
        "it's": "it is", "let's": "let us", "ma'am": "madam", "mightn't": "might not",
        "mustn't": "must not", "needn't": "need not", "needn't": "need not", "shan't": "shall not",
        "she'd": "she would", "she'll": "she will", "she's": "she is", "shouldn't": "should not",
        "that's": "that is", "there's": "there is", "they'd": "they would", "they'll": "they will",
        "they're": "they are", "they've": "they have", "wasn't": "was not", "weren't": "were not",
        "what's": "what is", "what'll": "what will", "what're": "what are", "what've": "what have",
        "where's": "where is", "where've": "where have", "who's": "who is", "who'll": "who will",
        "who're": "who are", "who've": "who have", "why's": "why is", "why're": "why are",
        "why've": "why have", "won't": "will not", "wouldn't": "would not", "you'd": "you would",
        "you'll": "you will", "you're": "you are", "you've": "you have"
    }
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    
    # Remove punctuation using regular expressions
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Define English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Customize the stopwords list for sentiment analysis (add negations or important words)
    sentiment_important_words = {"not", "no", "very", "good", "bad", "excellent", "love", "hate", "great", "feel", "wish", "would", "should"}
    stop_words = stop_words - sentiment_important_words  # Remove sentiment important words from stopwords

    # Function to map POS tag to wordnet format
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # Default to NOUN if unknown

    # POS tagging
    pos_tags = pos_tag(tokens)
    
    # Apply lemmatization based on POS tags and filter out stopwords
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags
        if word.isalnum() and word not in stop_words
    ]
    
    # Join the lemmatized tokens back into a single string
    return ' '.join(lemmatized_tokens)

# Test the function
df['processed_content'] = df['content'].apply(preprocess_text)
df


In [15]:
import random
import time
from sklearn.utils import resample
from deep_translator import GoogleTranslator
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # Import tqdm for progress bar
import pandas as pd

# Global variable to track the number of requests made today
requests_today = 0
MAX_REQUESTS_PER_DAY = 200000
REQUESTS_PER_SECOND = 5
SECONDS_PER_REQUEST = 1 / REQUESTS_PER_SECOND

# Counters for tracking errors and successes
success_count = 0
error_count = 0

# Function to perform back-translation and preprocessing with error handling
def back_translate_batch(texts, src_lang, intermediate_lang1, intermediate_lang2):
    global requests_today, success_count, error_count
    
    if requests_today >= MAX_REQUESTS_PER_DAY:
        print("Daily request limit reached!")
        error_count += len(texts)  # Count all texts as errors if the limit is reached
        return [preprocess_text(text) for text in texts]  # Return original texts if request limit is reached

    try:
        # Translate to the first intermediate language
        translated1 = GoogleTranslator(source=src_lang, target=intermediate_lang1).translate_batch(texts)
        
        # Translate to the second intermediate language
        translated2 = GoogleTranslator(source=intermediate_lang1, target=intermediate_lang2).translate_batch(translated1)
        
        # Translate back to the source language
        back_translated = GoogleTranslator(source=intermediate_lang2, target=src_lang).translate_batch(translated2)
        
        # Increment request counter
        requests_today += 3 * len(texts)  # One request for each translation stage (3 translations per text)
        
        time.sleep(SECONDS_PER_REQUEST * len(texts))  # Ensure we don't exceed the requests per second limit
        
        success_count += len(texts)  # Increment success counter
        return [preprocess_text(text) for text in back_translated]
    
    except Exception as e:
        # If any translation fails, log the error and return the original texts
        # print(f"Error during translation: {e}")
        error_count += len(texts)  # Increment error counter
        return [preprocess_text(text) for text in texts]  # Return the original texts if translation fails

# Parallelized function to oversample with back-translation and show progress bar
def oversample_with_back_translation(samples, oversample_count, sentiment_label, src_lang='en', intermediate_lang1='es', intermediate_lang2='fr'):
    global success_count, error_count
    
    # Create new samples using back-translation
    new_samples = []
    
    # Wrap the loop with ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        
        # Wrap the range of oversampling count with tqdm to show progress
        with tqdm(total=oversample_count, desc=f'Oversampling {sentiment_label}', unit='sample') as pbar:
            # Submit tasks to executor
            for _ in range(0, oversample_count, 100):  # Process in batches of 100
                batch_size = min(100, oversample_count - _)
                random_samples = random.choices(samples['content'].tolist(), k=batch_size)  # Choose random samples
                futures.append(executor.submit(back_translate_batch, random_samples, src_lang, intermediate_lang1, intermediate_lang2))
            
            # Collect the results as they finish and update the progress bar
            for future in futures:
                translated_texts = future.result()  # This will block until the task is finished
                for text in translated_texts:
                    new_samples.append({
                        'content': text,
                        'sentiment': sentiment_label
                    })
                pbar.update(len(translated_texts))  # Update progress bar after each completed batch
                
                # Update progress bar description with success and error percentages
                total_processed = success_count + error_count
                if total_processed > 0:
                    success_percentage = (success_count / total_processed) * 100
                    error_percentage = (error_count / total_processed) * 100
                    pbar.set_description(f"Oversampling {sentiment_label} | Success: {success_percentage:.2f}% | Errors: {error_percentage:.2f}%")
    
    # Convert new samples to a DataFrame
    new_samples_df = pd.DataFrame(new_samples)
    
    return new_samples_df

# Assuming df is your original DataFrame
# Separate the data by sentiment
negative_samples = df[df['sentiment'] == 'negative']
neutral_samples = df[df['sentiment'] == 'neutral']
positive_samples = df[df['sentiment'] == 'positive']

# Define target size (matching the negative class size)
target_size = len(negative_samples)

# Calculate oversample count for each class
neutral_oversample_count = max(target_size - len(neutral_samples), 0)
positive_oversample_count = max(target_size - len(positive_samples), 0)

# Oversample each class with back-translation
neutral_oversampled = oversample_with_back_translation(neutral_samples, neutral_oversample_count, 'neutral')
positive_oversampled = oversample_with_back_translation(positive_samples, positive_oversample_count, 'positive')

# Combine the oversampled classes to create a more balanced dataset
balanced_df = pd.concat([negative_samples, neutral_oversampled, positive_oversampled])

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the class balance
print(balanced_df['sentiment'].value_counts())  # Check class balance

# Print final success and error counts
total_samples = success_count + error_count
if total_samples > 0:
    success_percentage = (success_count / total_samples) * 100
    error_percentage = (error_count / total_samples) * 100
    print(f"\nFinal Translation Results:")
    print(f"Success: {success_percentage:.2f}% ({success_count} samples)")
    print(f"Errors: {error_percentage:.2f}% ({error_count} samples)")
else:
    print("\nNo translations were performed.")

Oversampling neutral | Success: 0.00% | Errors: 100.00%: 100%|██████████| 36887/36887 [01:17<00:00, 474.53sample/s]
Oversampling positive | Success: 0.00% | Errors: 100.00%: 100%|██████████| 25666/25666 [00:57<00:00, 445.60sample/s]

sentiment
negative    48906
neutral     36887
positive    25666
Name: count, dtype: int64

Final Translation Results:
Success: 0.00% (0 samples)
Errors: 100.00% (62553 samples)


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# Count the number of occurrences for each sentiment category
sentiment_count = balanced_df['sentiment'].value_counts()

plt.figure(figsize=(8, 8))  # Set the figure size

# Create a pie chart to visualize sentiment distribution
plt.pie(sentiment_count, labels=sentiment_count.index, autopct="%1.1f%%", startangle=140)

plt.title('Sentiment Distribution')  # Add a title to the chart
plt.axis('equal')  # Ensure the pie chart is displayed as a circle
plt.show()  # Display the plot

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(balanced_df['processed_content'], balanced_df['sentiment'], test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer with bigrams, minimum document frequency of 3, and max document frequency of 90%
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, sublinear_tf=True)

# Fit the vectorizer on training data and transform it into numerical feature vectors
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the test data using the fitted vectorizer
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_vectorized, y_train)
preds = rf.predict(X_test_vectorized)
print(classification_report(y_test, preds))